In [ ]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')

PROJECT_ROOT = '/content/drive/My Drive/trend_detection_ml'

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

os.chdir(PROJECT_ROOT)

print(f"✅ Current Working Directory: {os.getcwd()}")
print(f"✅ Files in Project Root: {os.listdir()}")

In [ ]:
from src.data_pipeline import load_data, validate_data
import matplotlib.pyplot as plt

df = load_data("BTC-USD", period="7d", interval="15m")

validate_data(df)

print(df.head())

plt.figure(figsize=(10, 5))
plt.plot(df['close'], label='BTC Price')
plt.title("Bitcoin Price (Last 7 Days)")
plt.legend()
plt.show()

In [ ]:
import importlib
import src.features

importlib.reload(src.features)

from src.features import create_features

df_processed = create_features(df)

print("\n--- New Features ---")
print(df_processed[['close', 'ma_10', 'rsi', 'target']].tail())

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 3))
plt.plot(df_processed.index, df_processed['rsi'], color='purple', label='RSI')
plt.axhline(70, linestyle='--', color='red', alpha=0.5)
plt.axhline(30, linestyle='--', color='green', alpha=0.5)
plt.title("RSI Indicator (Momentum)")
plt.legend()
plt.show()

In [ ]:
import importlib
import src.model
importlib.reload(src.model)
from src.model import CryptoModel

features = ['ma_10', 'ma_50', 'rsi', 'volatility', 'log_return']
target = 'target'

X = df_processed[features]
y = df_processed[target]

split_point = int(len(X) * 0.8)

X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

print(f"Training on {len(X_train)} rows, Testing on {len(X_test)} rows.")

bot = CryptoModel(n_estimators=100, max_depth=3)
bot.train(X_train, y_train)

bot.evaluate(X_test, y_test)

save_path = '/content/drive/My Drive/trend_detection_ml/models/xgboost_v1.joblib'
bot.save(save_path)